<a href="https://colab.research.google.com/github/rmatrella/BirdsSound/blob/main/Utilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive/')
%cd "/content/drive/MyDrive/Marino_Matrella"

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1pfBa0fnatsy0qYLtZ2wpGwzSR-zOPP2W/CIDL project


In [ ]:
! pip install -U tensorflow_addons
! pip install -q "tqdm>=4.36.1"
!pip install spec_augment
!pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 29.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.0/441.0 kB 47.0 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3


In [ ]:
import os
import zipfile
import random
import pickle
import pandas as pd
import numpy as np
import tqdm
import shutil
from PIL import Image

from google.colab import drive

from sklearn import metrics
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

import tensorflow as tf
from tensorflow import keras
#import tensorflow_addons as tfa
from keras import regularizers
from tensorflow.keras import layers, models, metrics
from tensorflow.keras.metrics import Precision, Recall, F1Score
from tensorflow.keras.utils import to_categorical, image_dataset_from_directory
from tensorflow.keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow import keras
from spec_augment import SpecAugment
import cv2 as cv2
from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
import matplotlib.cm as cm

sns.set_theme(style='ticks')

In [ ]:
print('tensor version is ', tf.__version__)

tensor version is  2.14.0-dev20230613


In [ ]:
def compute_class_weights(train_path):
  labels = os.listdir(train_path)
  classes = []
  for label in labels:
    n = len(os.listdir(os.path.join(train_path,label)))
    p = [label] * n
    classes = [y for x in [classes, p] for y in x]

  from sklearn.utils.class_weight import compute_class_weight
  class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(classes), y= np.array(classes))
  class_weights={k: v for k, v in enumerate(class_weights)}

  return class_weights

In [ ]:
def set_seed ():
  '''
  set_seed is used to obtain reproducible results using keras during the development phase
  '''
  seed = 24
  # The below is necessary for reproducible results of certain Python hash-based operations.
  os.environ["PYTHONHASHSEED"]="0"
  # The below is necessary for starting Numpy generated random numbers in a well-defined initial state.
  np.random.seed(seed)
  # The below is necessary for starting core Python generated random numbers in a well-defined state.
  random.seed(seed)
  # The below tf.random.set_seed will make random number generation in TensorFlow have a well-defined initial state.
  tf.random.set_seed(seed)

In [ ]:
set_seed()

In [ ]:
def compile_model (model, optimizer='adam', learning_rate = 0.0001):
  '''
  compile_model is used to compile the current model
  :param model: model to compile
  :param optimizer: optimizer to be used
  :param learning_rate: learning rate parameter for the optimizer
  '''
  if optimizer == 'adam':
    model.compile(loss = "categorical_crossentropy",
                  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics = [
                              "accuracy",
                              F1Score(average="weighted")
                            ])

  elif optimizer == 'rmsprop':
    model.compile(loss = "categorical_crossentropy",
                  optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
                  metrics = [
                              "accuracy",
                              F1Score(average="weighted")
                            ])
  elif optimizer == 'sgd':
    model.compile(loss = "categorical_crossentropy",
                  optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, weight_decay=0.001),
                  metrics = [
                              "accuracy",
                              F1Score(average="weighted")
                            ])

  return model


def run_model (model, model_name, train_dataset, validation_dataset, MODELS_FOLDER, class_weights=None, epochs=50, patience=5, monitor='val_loss'):    #class_weights=None
  '''
  run_model is used to run the current mode
  :param model: model to run
  :param model_name: name given to save the model
  :param epochs: how many epochs to do
  :param patience: patience value for Early Stopping
  :param monitor: what to monitor for Early Stopping and Model Checkpoint
  '''
  if not os.path.exists(MODELS_FOLDER):
    os.makedirs(MODELS_FOLDER)

  #save path for the models
  checkpoint_filepath = os.path.join(MODELS_FOLDER, model_name + '.h5')

  callbacks_list = [
                  keras.callbacks.EarlyStopping(
                      monitor=monitor,
                      patience=patience,
                      verbose=1,
                      mode='min'
                  ),
                  keras.callbacks.ModelCheckpoint(
                      filepath = checkpoint_filepath,
                      monitor=monitor,
                      verbose=1,
                      save_best_only=True
                  ),
                  keras.callbacks.ReduceLROnPlateau(
                      monitor=monitor,
                      factor=0.1,
                      patience=3
                  ),
                  # tfa.callbacks.TQDMProgressBar()
                  ]


  history = model.fit(train_dataset,
                    epochs=epochs,
                    validation_data=validation_dataset,
                    callbacks=callbacks_list,
                    class_weight=class_weights
                    )

  history = pd.DataFrame(history.history)
  history.to_csv(os.path.join(MODELS_FOLDER, model_name + '_history.csv'), index=False)

  return history


def plot_model_performance(history=None, path=None, axis=1):
  '''
  show training and validation performances, plotting the performances during the training phase
    :param history: object in which are recorded all the events
    :path current path of the hystory saved somewhere
    :axis vertical (1) or horizontal (0) plot
  '''

  if path is not None:
    history = pd.read_csv(path + '_history.csv')

  s = pd.Index(np.arange(history.shape[0])+1)
  accuracy_results = history[['accuracy', 'val_accuracy']]
  accuracy_results = accuracy_results.set_index([s])

  loss_results = history[['loss', 'val_loss']]
  loss_results = loss_results.set_index([s])
  val_loss_min = loss_results[['val_loss']].idxmin()
  val_loss_min=val_loss_min.values[0]

  f1_results = history[['f1_score', 'val_f1_score']]
  f1_results = f1_results.set_index([s])

  if axis == 1:
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(5,10))
  else:
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,4))

  sns.lineplot(data = accuracy_results, ax=ax1)
  ax1.title.set_text('Training and validation accuracy')
  ax1.set_xlabel('Epochs')
  ax1.set_ylabel('Accuracy')
  ax1.axvline(val_loss_min, color='lightgrey')
  ax1.grid()

  print()

  sns.lineplot(data = loss_results, ax=ax2)
  ax2.title.set_text('Training and validation loss')
  ax2.set_xlabel('Epochs')
  ax2.set_ylabel('Loss')
  ax2.axvline(val_loss_min, color='lightgrey')
  ax2.grid()

  print()

  sns.lineplot(data = f1_results, ax=ax3)
  ax3.title.set_text('Training and validation f1')
  ax3.set_xlabel('Epochs')
  ax3.set_ylabel('F1')
  ax3.axvline(val_loss_min, color='lightgrey')
  ax3.grid()


def evaluate_model(model):  ###ok

  y_true = tf.concat([labels_batch for data_batch, labels_batch in test_set], axis = 0)
  y_true_cat = np.argmax(y_true, -1)

  pred_Y = model.predict(test_set, verbose = True)
  pred_Y_cat = np.argmax(pred_Y, -1)

  # print metrics
  results=model.evaluate(test_set)
  print()
  print(f"Test loss: {results[0]:.3f}")
  print(f"Test accuracy: {results[1]:.3f}")
  print(f"Test f1: {results[2]:.3f}")
  print()

  accuracy = accuracy_score(y_true_cat, pred_Y_cat)
  print("Accuracy on test data: %.2f %%" % (accuracy * 100))
  f1 = f1_score(y_true_cat, pred_Y_cat, average='weighted')
  print("F1-Score on test data: %.2f %%" % (f1 * 100))
  print()
  class_label = np.array(test_set.class_names)
  print(classification_report(class_label[y_true_cat], class_label[pred_Y_cat]))

In [ ]:

spec_augment = keras.Sequential([

#  LB usato
  SpecAugment(freq_mask_param=27,
                           time_mask_param=100,
                           n_freq_mask=1,
                           n_time_mask=1
                           )
  #LD
    # SpecAugment(freq_mask_param=27,
    #                        time_mask_param=100,
    #                        n_freq_mask=2,
    #                        n_time_mask=2,
    #                        mask_value=-100)
  #SM usato
  # SpecAugment(freq_mask_param=15,
  #                          time_mask_param=70,
  #                          n_freq_mask=2,
  #                          n_time_mask=2,
  #                          mask_value=-100)
  #SS
  #  SpecAugment(freq_mask_param=27,
  #                          time_mask_param=70,
  #                          n_freq_mask=2,
  #                          n_time_mask=2,
  #                          mask_value=-100)
])


In [ ]:
data_augmentation = keras.Sequential(
  [
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomContrast(factor=0.15),
  layers.RandomZoom(0.1, fill_mode="constant", fill_value=0.0),
  layers.RandomRotation(factor=1, fill_mode="constant", fill_value=0.0),
  ]
)

In [ ]:
def get_class_labels():
  return np.array(test_set.class_names)